In [15]:
#code from https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
import requests
import pandas as pd
from datetime import datetime

# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

with open('pw.txt', 'r') as f:
    pw = f.read()

# authenticate API
client_auth = requests.auth.HTTPBasicAuth('ZLJQUDUpM5tgeaJJ_MfbjA', 'JwKa8UDlUr_KxPDkrmQF1AJuFrzysg')
data = {
    'grant_type': 'password',
    'username': 'docmouseMcFlavor',
    'password': pw
}
headers = {'User-Agent': 'myBot/0.0.1'}

# send authentication request for OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=data, headers=headers)
# extract token from response and format correctly
token = f"bearer {res.json()['access_token']}"
# update API headers with authorization (bearer token)
headers = {**headers, **{'Authorization': token}}

# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(3):
    # make request
    res = requests.get("https://oauth.reddit.com/r/wallstreetbets/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)


In [16]:
data

,subreddit,title,selftext,upvote_ratio,ups,downs,score,link_flair_css_class,created_utc,id,kind
0,wallstreetbets,Why something as simple as using the same lot ...,,1.00,1.0,0.0,1.0,chart,2022-07-11T06:12:29Z,vwi4jc,t3
1,wallstreetbets,parabolic! NG etf derivatives are going to be ...,,1.00,1.0,0.0,1.0,chart,2022-07-11T06:10:57Z,vwi3g7,t3
2,wallstreetbets,AMC Theatres Saw Record Ticket Sales This Weekend,[Yahoo News Article](https://finance.yahoo.com...,0.75,2.0,0.0,2.0,question,2022-07-11T06:10:19Z,vwi2ze,t3
3,wallstreetbets,How to Trade CPI day,&amp;#x200B;\n\nhttps://preview.redd.it/9wfhc2...,1.00,5.0,0.0,5.0,dd,2022-07-11T06:01:05Z,vwhw7o,t3
4,wallstreetbets,"Take a nap Jimmy, you look like you drink four...",,0.85,14.0,0.0,14.0,meme,2022-07-11T05:56:06Z,vwhs8c,t3
...,...,...,...,...,...,...,...,...,...,...,...
295,wallstreetbets,Snap hires the head of US Secret Service,,0.82,7.0,0.0,7.0,meme,2022-07-07T23:58:23Z,vu4jo0,t3
296,wallstreetbets,Euro Yolo?,So the euro is at an all time low rn. Do I con...,1.00,6.0,0.0,6.0,question,2022-07-07T23:25:34Z,vu41dg,t3
297,wallstreetbets,So I got margin called for $21.5k.,So I got a day-trade margin / day trade min eq...,0.83,33.0,0.0,33.0,loss,2022-07-07T23:23:54Z,vu40e9,t3
298,wallstreetbets,Emperor is naked,Bulls dont realize the mkt right now is empero...,0.71,25.0,0.0,25.0,question,2022-07-07T22:48:02Z,vu3fni,t3
